In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu Mar 21 21:30:27 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.819
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


#### Set variables

In [2]:
%env download_dir=/home/sam/data/metagenomics/P_generosa
%env blastdb_dir=/home/sam/data/blast_dbs
%env blastdb=swissprot_v5
%env blastdb_url=https://ftp.ncbi.nlm.nih.gov/blast/db/v5/swissprot_v5.tar.gz
%env fasta_url=http://gannet.fish.washington.edu/Atumefaciens/20190103_metagenomics_geo_metagenemark/20190103-mgm-proteins.fa
%env blastp_out_dir=/home/sam/analyses/20190321_metagenomics_pgen_blastp
%env input_fasta=/home/sam/data/metagenomics/P_generosa/20190103-mgm-proteins.fa
%env blastp_out=blastp.outfmt6
%env blastp=/home/sam/programs/ncbi-blast-2.8.1+/bin/blastp
%env threads=23

env: download_dir=/home/sam/data/metagenomics/P_generosa
env: blastdb_dir=/home/sam/data/blast_dbs
env: blastdb=swissprot_v5
env: blastdb_url=https://ftp.ncbi.nlm.nih.gov/blast/db/v5/swissprot_v5.tar.gz
env: fasta_url=http://gannet.fish.washington.edu/Atumefaciens/20190103_metagenomics_geo_metagenemark/20190103-mgm-proteins.fa
env: blastp_out_dir=/home/sam/analyses/20190321_metagenomics_pgen_blastp
env: input_fasta=/home/sam/data/metagenomics/P_generosa/20190103-mgm-proteins.fa
env: blastp_out=blastp.outfmt6
env: blastp=/home/sam/programs/ncbi-blast-2.8.1+/bin/blastp
env: threads=23


#### Make directories

```--parents``` option allows creation of multiple, nested directories

In [3]:
%%bash
mkdir --parents ${blastp_out_dir}
mkdir --parents ${blastdb_dir}
ls -l ${blastp_out_dir}
ls -l ${blastdb_dir}

total 0
-rw-rw-r-- 1 sam sam 0 Mar 21 18:59 blastp.outfmt6
total 0


#### Transfer metagenomics FastA

In [4]:
%%bash
rsync \
--archive \
--verbose \
--progress \
gannet:/volume2/web/Atumefaciens/20190103_metagenomics_geo_metagenemark/20190103-mgm-proteins.fa \
${download_dir}
    
#Uncomment below to download via web URL
# Will need to set desired path with --directory-prefix
#wget \
#--directory-prefix=${download_dir} \
#--quiet \
#${fasta_url}

echo "-------------------------"
ls -lh ${download_dir}

receiving incremental file list

sent 11 bytes  received 69 bytes  160.00 bytes/sec
total size is 762,710,574  speedup is 9,533,882.18
-------------------------
total 4.5G
-rw-r--r-- 1 sam users 1.6G Jan  3 14:51 20190103-mgm-nucleotides.fa
-rw-r--r-- 1 sam users 728M Jan  3 14:51 20190103-mgm-proteins.fa
-rw-r--r-- 1 sam users 2.2G Jan  3 06:08 final.contigs.fa


#### Run BLASTp

In [5]:
%%bash
wget \
--directory-prefix ${blastdb_dir} \
--quiet \
${blastdb_url}

tar -C ${blastdb_dir} -xzvf ${blastdb_dir}/${blastdb}.tar.gz

echo "-----------------------------"
ls -lh ${blastdb_dir}

swissprot_v5.pin
swissprot_v5.phr
swissprot_v5.psq
swissprot_v5.ppi
swissprot_v5.ppd
swissprot_v5.pog
swissprot_v5.pdb
swissprot_v5.pdb
swissprot_v5.pos
swissprot_v5.pot
swissprot_v5.ptf
swissprot_v5.pto
taxdb.btd
taxdb.bti
-----------------------------
total 646M
-rw-r--r-- 1 sam sam  23M Feb  7 17:16 swissprot_v5.pdb
-rw-r--r-- 1 sam sam 114M Feb  7 17:16 swissprot_v5.phr
-rw-r--r-- 1 sam sam 3.6M Feb  7 17:16 swissprot_v5.pin
-rw-r--r-- 1 sam sam 1.8M Feb  7 17:16 swissprot_v5.pog
-rw-r--r-- 1 sam sam 8.4M Feb  7 17:16 swissprot_v5.pos
-rw-r--r-- 1 sam sam 5.8M Feb  7 17:16 swissprot_v5.pot
-rw-r--r-- 1 sam sam 3.6M Feb  7 17:16 swissprot_v5.ppd
-rw-r--r-- 1 sam sam  15K Feb  7 17:16 swissprot_v5.ppi
-rw-r--r-- 1 sam sam 170M Feb  7 17:16 swissprot_v5.psq
-rw-r--r-- 1 sam sam 436K Feb  7 17:16 swissprot_v5.ptf
-rw-r--r-- 1 sam sam 2.2M Feb  7 17:16 swissprot_v5.pto
-rw-rw-r-- 1 sam sam 172M Feb 19 08:58 swissprot_v5.tar.gz
-rw-rw-r-- 1 sam sam 129M Feb 19 08:58 taxdb.btd
-rw-rw-r-- 

In [8]:
%%bash

cd ${blastdb_dir}

export BLASTDB=${blastdb_dir}

time \
${blastp} \
-query ${input_fasta} \
-db ${blastdb} \
-evalue 1E-04 \
-outfmt "6 std staxids" \
-max_target_seqs 1 \
-num_threads ${threads} \
-out ${blastp_out_dir}/${blastp_out} \
1> stdout.txt \
2> stderr.txt


real	3877m13.548s
user	85954m17.252s
sys	300m8.416s
